In [1]:
pip install pyspark

     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 54.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=f5b0586d96b2ec0991e57ff5267083f3e1bda051ea43dbecfbc76eae7e921efb
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
from pyspark.ml.clustering import KMeans

In [4]:
from pyspark.sql import SparkSession

In [5]:
session = SparkSession.builder.appName('cluster').getOrCreate()

In [6]:
data = session.read.csv("cluster menu.csv", header = True, inferSchema=True)

In [7]:
data.show(5)

+---------+--------------------+--------------+--------+-----------------+---------+-------------------------+-------------+-----------------------------+---------+-----------+---------------------------+------+----------------------+-------------+-----------------------------+-------------+-----------------------------+------+-------+-------------------------+-------------------------+-----------------------+--------------------+
| Category|                Item|  Serving Size|Calories|Calories from Fat|Total Fat|Total Fat (% Daily Value)|Saturated Fat|Saturated Fat (% Daily Value)|Trans Fat|Cholesterol|Cholesterol (% Daily Value)|Sodium|Sodium (% Daily Value)|Carbohydrates|Carbohydrates (% Daily Value)|Dietary Fiber|Dietary Fiber (% Daily Value)|Sugars|Protein|Vitamin A (% Daily Value)|Vitamin C (% Daily Value)|Calcium (% Daily Value)|Iron (% Daily Value)|
+---------+--------------------+--------------+--------+-----------------+---------+-------------------------+-------------+------

In [8]:
data.count()

260

In [9]:
data.columns

['Category',
 'Item',
 'Serving Size',
 'Calories',
 'Calories from Fat',
 'Total Fat',
 'Total Fat (% Daily Value)',
 'Saturated Fat',
 'Saturated Fat (% Daily Value)',
 'Trans Fat',
 'Cholesterol',
 'Cholesterol (% Daily Value)',
 'Sodium',
 'Sodium (% Daily Value)',
 'Carbohydrates',
 'Carbohydrates (% Daily Value)',
 'Dietary Fiber',
 'Dietary Fiber (% Daily Value)',
 'Sugars',
 'Protein',
 'Vitamin A (% Daily Value)',
 'Vitamin C (% Daily Value)',
 'Calcium (% Daily Value)',
 'Iron (% Daily Value)']

In [18]:
data.describe()

DataFrame[summary: string, Category: string, Item: string, Serving Size: string, Calories: string, Calories from Fat: string, Total Fat: string, Total Fat (% Daily Value): string, Saturated Fat: string, Saturated Fat (% Daily Value): string, Trans Fat: string, Cholesterol: string, Cholesterol (% Daily Value): string, Sodium: string, Sodium (% Daily Value): string, Carbohydrates: string, Carbohydrates (% Daily Value): string, Dietary Fiber: string, Dietary Fiber (% Daily Value): string, Sugars: string, Protein: string, Vitamin A (% Daily Value): string, Vitamin C (% Daily Value): string, Calcium (% Daily Value): string, Iron (% Daily Value): string]

In [19]:
from pyspark.ml.feature import VectorAssembler

In [20]:
vector_ass = VectorAssembler(inputCols = [
'Calories',
'Calories from Fat',
'Total Fat',
'Total Fat (% Daily Value)',
'Saturated Fat',
'Saturated Fat (% Daily Value)',
'Trans Fat',
'Cholesterol',
'Cholesterol (% Daily Value)',
'Sodium',
'Sodium (% Daily Value)',
'Carbohydrates',
'Carbohydrates (% Daily Value)',
'Dietary Fiber',
'Dietary Fiber (% Daily Value)',
'Sugars',
'Protein',
'Vitamin A (% Daily Value)',
'Vitamin C (% Daily Value)',
'Calcium (% Daily Value)',
'Iron (% Daily Value)'], outputCol = "allfeatures")

In [21]:
newdata = vector_ass.transform(data)

In [22]:
newdata.show(5)

+---------+--------------------+--------------+--------+-----------------+---------+-------------------------+-------------+-----------------------------+---------+-----------+---------------------------+------+----------------------+-------------+-----------------------------+-------------+-----------------------------+------+-------+-------------------------+-------------------------+-----------------------+--------------------+--------------------+
| Category|                Item|  Serving Size|Calories|Calories from Fat|Total Fat|Total Fat (% Daily Value)|Saturated Fat|Saturated Fat (% Daily Value)|Trans Fat|Cholesterol|Cholesterol (% Daily Value)|Sodium|Sodium (% Daily Value)|Carbohydrates|Carbohydrates (% Daily Value)|Dietary Fiber|Dietary Fiber (% Daily Value)|Sugars|Protein|Vitamin A (% Daily Value)|Vitamin C (% Daily Value)|Calcium (% Daily Value)|Iron (% Daily Value)|         allfeatures|
+---------+--------------------+--------------+--------+-----------------+---------+----

In [34]:
model = KMeans(featuresCol = "allfeatures").setK(3)
new_model = model.fit(newdata)
result = new_model.transform(newdata)
result.show()

+---------+--------------------+--------------+--------+-----------------+---------+-------------------------+-------------+-----------------------------+---------+-----------+---------------------------+------+----------------------+-------------+-----------------------------+-------------+-----------------------------+------+-------+-------------------------+-------------------------+-----------------------+--------------------+--------------------+----------+
| Category|                Item|  Serving Size|Calories|Calories from Fat|Total Fat|Total Fat (% Daily Value)|Saturated Fat|Saturated Fat (% Daily Value)|Trans Fat|Cholesterol|Cholesterol (% Daily Value)|Sodium|Sodium (% Daily Value)|Carbohydrates|Carbohydrates (% Daily Value)|Dietary Fiber|Dietary Fiber (% Daily Value)|Sugars|Protein|Vitamin A (% Daily Value)|Vitamin C (% Daily Value)|Calcium (% Daily Value)|Iron (% Daily Value)|         allfeatures|prediction|
+---------+--------------------+--------------+--------+----------

In [35]:
cluster1 = result.filter(result["prediction"]== 0)
cluster1.count()

181

In [36]:
cluster2 = result.filter(result["prediction"]==1)
cluster2.count()

78

In [37]:
cluster3 = result.filter(result["prediction"]==2)
cluster3.count()

1